In [1]:
import matplotlib, cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import glob
import math
import time
from scipy import ndimage
%matplotlib inline

In [2]:
def show(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))

In [3]:
def load_training_images(n):
        images = np.full([n], None)
        names = np.full([n], str)
        i = 0
        for name in glob.glob('dataset1/training/png/*.png'):
            images[i] = cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB)
            names[i] = name[26:len(name)-4]
            i += 1
        return [images, names]

In [4]:
def generate_pyramids(templates):
    n = len(templates)
    octaves = 3
    spo = 2 # scales per octave
    nscales = (octaves * spo) + 1
    template_size = 512
    scale_factor = (1 /(2 ** (1 / spo))) 
    sigma = round(scale_factor / math.pi, 2)
    pyramids = np.full((n), np.array)
    
    for i in range(n):
        pyramid = np.full((nscales), np.array)
        for j in range(nscales):
            if j == 0:
                pyramid[0] = cv2.cvtColor(templates[i], cv2.COLOR_RGB2GRAY)
                size = template_size
            else:
                size = round(size * scale_factor)
                temp = cv2.GaussianBlur(pyramid[j - 1], [0, 0], sigmaX = sigma)
                pyramid[j] = cv2.resize(temp, [size, size])
        pyramids[i] = pyramid
        
    return pyramids

In [5]:
def slowsimilarity(img1, img2):
    if(not len(img1) == len(img2) or not len(img1[0]) == len(img2[0])):
        print("Incompatible sizes")
        return
    acc = 0
    for i in range(len(img1)):
        for j in range(len(img1[0])):
            acc += abs(int(img1[i][j]) - int(img2[i][j]))
    return acc / (len(img1) ** 2)

In [6]:
def quicksimilarity(img1, img2):
    quality = 6
    if(not len(img1) == len(img2) or not len(img1[0]) == len(img2[0])):
        print("Incompatible sizes")
        return
    acc = 0
    for i in range(quality):
        for j in range(quality):
            x = int((i / quality) * len(img1))
            y = int((j / quality) * len(img1[0]))
            acc += abs(int(img1[x][y]) - int(img2[x][y]))
    return acc / (quality ** 2)

In [7]:
def detect_image(template, testimage):
    nscales = len(template)
    basesize = len(testimage)
    bestscore = 256
    xpos = []
    ypos = []
    scores = []
    templates = []
    candsize = basesize
    skipnext = False
    
    for s in range(nscales):
        candsize = len(template[s])
        if(candsize > 100): dscale = 8
        elif(candsize > 64): dscale = 4
        elif(candsize > 40): dscale = 2
        else: dscale = 1
        if(candsize == basesize):
            score = quicksimilarity(template[s], testimage)
            if(score < bestscore):
                bestscore = score
        else:
            newsize = round(candsize / dscale)
            testimgsize = round(basesize / dscale)
            X = round((basesize - candsize) / dscale)
            scaledtemplate = cv2.resize(template[s], [newsize, newsize])
            scaledtestimg = cv2.resize(testimage, [testimgsize, testimgsize])
            for x in range(X):
                for y in range(X):
                    if(skipnext):
                        skipnext = False
                        continue
                    score = quicksimilarity(scaledtemplate, scaledtestimg[y:(y + newsize), x:(x + newsize)])
                    #print(score)
                    if(score < 20):
                        xpos.append(dscale * x)
                        ypos.append(dscale * y)
                        scores.append(score)
                        templates.append(s)
                    if(score > 50):
                        skipnext = True
    #print("first = " + str(len(scores)))
    if(len(scores) > 200):
        return [[], None, 0, 0]
    #non-maximum suppression
    R = []
    for i in range(len(scores) - 1):
        for j in range(i + 1, len(scores)):
            maxlen = max(len(template[templates[i]]), len(template[templates[j]]))
            if(abs(xpos[i] - xpos[j]) < maxlen and 
                  abs(ypos[i] - ypos[j]) < maxlen):
                if(scores[i] < scores[j]):
                    R.append(j)
                else: R.append(i)
    R = set(R)
    R2 = list(R)
    R2.sort(reverse = True)
    for r in R2:
        del scores[r]
        del xpos[r]
        del ypos[r]
        del templates[r]
        
    #fine-tune
    nsolutions = len(scores)
    #print("second = " + str(nsolutions))
    if(nsolutions == 0):
        return [[], None, 0, 0]
    
    ftr = 2 #fine-tune range
    for s in range(nsolutions):
        
        bestsize = len(template[templates[s]])
        x = xpos[s]
        y = ypos[s]

        for i in range(-ftr, ftr):
            for j in range(-ftr, ftr):
                if(y + i < 0 or y + i + bestsize > basesize or x + j < 0 or x + j + bestsize > basesize):
                    continue
                if(not i == 0 or not j == 0):
                    score = quicksimilarity(template[templates[s]], testimage[y + i: y + i + bestsize, x + j: x + j + bestsize])
                    #print(score)
                    if(score < scores[s]):
                        scores[s] = score
                        ypos[s] = y + i
                        xpos[s] = x + j
         
    
    return [scores, templates, xpos, ypos]

In [8]:
def detect_images(names, pyramids, I):
    fig = plt.figure()
    ax = fig.gca()
    ntraining_images = 50
    colours = ["red", "blue", "green", "orange", "purple"]
    imgTest = cv2.cvtColor(cv2.imread("dataset1/test/images/test_image_" + str(I + 1) + ".png"), cv2.COLOR_BGR2RGB)
    j = 0
    for i in range(ntraining_images):
        iterate_colour = False
        [scores, t, xpos, ypos] = detect_image(pyramids[i], cv2.cvtColor(imgTest, cv2.COLOR_RGB2GRAY))
        for s in range(len(scores)):
            if(scores[s] < 8):
                ax.add_patch(plt.Rectangle([xpos[s], ypos[s]], len(pyramids[i][t[s]] + 1), len(pyramids[i][t[s]] + 1), fill = False, ec = colours[j], label = names[i]))
                iterate_colour = True
        if(iterate_colour):
            j += 1
            if(j == 5):
                j = 0
            
    
    ax.legend()
    ax.imshow(imgTest)
    canvas = FigureCanvas(fig)
    canvas.draw()
    buf = canvas.buffer_rgba()
    image = np.asarray(buf)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite("output_" + str(I + 1) + ".png", image)

In [10]:
start_time = time.time()
[templates, names] = load_training_images(50)
pyramids = generate_pyramids(templates)
for i in range(20):
    detect_images(names, pyramids, i)
    print(i)
print("--- %s seconds ---" % ((time.time() - start_time) / 20))

--- 0.007678043842315674 seconds ---
